In [1]:
import pandas as pd

In [3]:
##Read in csv 
met_df = pd.read_csv('D:\MetObjects.csv')

C:\Users\Leanne\Anaconda2\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Create a list of the columns we want to pull, based on the db schema created
columns = ['Department', 'Object Name', 'Title', 'Artist Alpha Sort', 'Artist Begin Date', 'Artist End Date', 'Object Begin Date', 'Object End Date', 'Medium', 'Classification', 'Link Resource']

#Create a dataframe that only contains 'painting' objects
painting_filter = met_df['Object Name'].isin(["Painting"]) 
paintings = pd.DataFrame(met_df[painting_filter], columns=columns)

#display counts to identify columns
#paintings.head(1)

In [5]:
#read out counts to identify columns with missing values - total is to have 5,887
paintings.count()

Department           5887
Object Name          5887
Title                5553
Artist Alpha Sort    5654
Artist Begin Date    5451
Artist End Date      5454
Object Begin Date    5887
Object End Date      5887
Medium               5887
Classification       5887
Link Resource        5887
dtype: int64

In [6]:
# 334 records are missing title, which we want to drop...so drop those. 
paintings = paintings.dropna(subset=['Title'])

In [7]:
#Some of the default entries for artist birth/date are standardized to '9999', which we'll want to replace with ours ('unknown')
#Then we discover that there are 'citation' credits that include multiple artists names and dates seperated by "|"
#We only want the first/primary artist, not the citation, so we'll split str and just dump the extra
paintings['Artist Alpha Sort'] = paintings['Artist Alpha Sort'].str.split('|', expand=True)
paintings['Artist Begin Date'] = paintings['Artist Begin Date'].str.split('|', expand=True)
paintings['Artist End Date'] = paintings['Artist End Date'].str.split('|', expand=True)

In [8]:
#Split out Artist Alpha Sort into first name and last name. 
#There were hidden characters, so we had to have two extra holding columns, then delete
paintings[['Artist_Surname', 'Artist_First_Name', 'Blank1', 'Blank2']] = paintings['Artist Alpha Sort'].str.split(",",expand=True)
paintings = paintings.drop(columns=['Artist Alpha Sort','Blank1', 'Blank2'])

In [9]:
#Still missing 111 artists and some other info, but we have all the important ones like object, medium, dept and dates and link
#So, we still want to keep these anonymous paintings; we'll fill the missing  with 'unknown'
paintings = paintings.fillna('Unknown')

In [10]:
#Now we will replace the '9999' for End dates (and begin dates too just in case) 
paintings['Artist End Date'] = paintings['Artist End Date'].str.replace('9999', 'Unknown')

In [12]:
paintings.to_csv('paintings_working_file.csv')

In [20]:
paintings.count()

Department           5553
Object Name          5553
Title                5553
Artist Begin Date    5553
Artist End Date      5553
Object Begin Date    5553
Object End Date      5553
Medium               5553
Classification       5553
Link Resource        5553
Artist_Surname       5553
Artist_First_Name    5553
dtype: int64

In [18]:
artists = pd.DataFrame(paintings, columns=['Artist_Surname','Artist_First_Name'])
artists = artists.drop_duplicates()
artists.count()

Artist_Surname       2317
Artist_First_Name    2317
dtype: int64

In [21]:
artists.to_csv('unique_artists_for_scrape.csv')

In [ ]:
artists['Artist_First_Name'] = artists['Artist_First_Name'].str.strip()